In [2]:
import sys
sys.path.append('../../')
import helper

from pandas import read_excel, DataFrame, merge
from matplotlib import pyplot as plt
import seaborn as sb
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import ModelCheckpoint


In [ ]:
boston = read_excel("boston.xlsx")
boston.info()

In [41]:
boston_df = DataFrame(boston,columns=boston.columns)
boston_df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV,CAT. MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0,0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6,0
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7,1
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4,1
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2,1


In [47]:
a = list(boston_df.columns)
a

['CRIM',
 'ZN',
 'INDUS',
 'CHAS',
 'NOX',
 'RM',
 'AGE',
 'DIS',
 'RAD',
 'TAX',
 'PTRATIO',
 'B',
 'LSTAT',
 'MEDV',
 'CAT. MEDV']

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score # 성능 측정 평가 지표 

# x,y 분할
y = boston_df["PRICE"]
x = boston_df.drop(['PRICE'],axis = 1, inplace = False)

# 훈련용 데이터와 평가용 데이터 분할하기
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3,random_state = 156)

# 선형회귀 분석 모델 생성 
lr = LinearRegression()

# 선형회귀 분석 : 모델 훈련 (학습데이터를 가지고 수행)
lr.fit(x_train, y_train)


#선형회귀 분석 : 평가데이터(x_test) 에 대한 예측 후생 -> 예측 결과 y_predict 구하기 
y_predict = lr.predict(x_test)


# 선형회귀 분석 모델을 평가지표를 통해 평가하고 회귀 계수를 확인하여 피처의 영향을 분석한다.

mse = mean_squared_error(y_test,y_predict)
rmse = np.sqrt(mse)
print('mse:{0:.3f}, RMSE:{1:.3f}'.format(mse,rmse))
print('R^2(Variance score) : {0:.3f}'.format(r2_score(y_test,y_predict)))

# 회귀 분석은 지도 학습이므로 평가 데이터 x 에 대한 결과값 y를 이미 알고 있는 상태에서 평가 데이터 y_test와 위에서 구한 예측 결과 y_predict의 오차를 계산하여 모델을 평가한다. 평가 지표 MSE를 구하고 구한 값의 제곱근을 계산하여 모델을 평가한다. 평가 지표 MSE를 구하고 구한 값의 제곱근을 계산하여 np.sqrt 평가 지표 RMSE를 구한다. 그리고 평가 지표 R2를 구한다 (r2_score)

print('y 절편 값:',lr.intercept_)
print('회귀 계수 값 : ',np.round(lr.coef_,1))

# 선형회귀의 y절편과 각 피처의 회귀 계수를 확인한다. 

coef = Series(data=np.round(lr.coef_,2),index = x.colums)
coef.sort_values(ascending=False)
# 회귀 모델에서 구한 회귀 계수 값 lr.coef_과 피처 이름x.coulums를 묶어서 시리즈 자료형으로 만들고, 회귀 계수 값을 기준으로 내림차순으로 정렬하여 ascending 확인한다. 


# 회귀 분석 결과를 산점도 + 선형 회귀 그래프로 시각화 하기

import matplotilb.pyplot as plt
import seaborn as sns

fig, axs = plt.subplots(figsize = (16,16), ncols = 3, nrows=5)
x_features = ['CRIM',
 'ZN',
 'INDUS',
 'CHAS',
 'NOX',
 'RM',
 'AGE',
 'DIS',
 'RAD',
 'TAX',
 'PTRATIO',
 'B',
 'LSTAT'] # list(boston_df.columns)

for i, feature in enumerate(x_features) : 
    row = int(i/3)
    col = i%3
    sb.regplot(x=feature, y='PRICE', data = boston_df, ax = axs[row][col])

# 독립 변수인 13개 피처와 종속 변수인 주택 가격 PRICE와의 회귀관계를 보여주는 13개 그래프를 subplots()를 사용하여 5행 3열 구조로 모아서 나타낸다.
# seaborn의 regplot()은 산점도 그래프와 선형 회귀 그래프를 함께 그려준다. 

# 로지스틱 회귀 분석

로지스틱 회귀는 분류에 사용하는 기법으로 선형 함수를 사용하여 추세를 예측하는 선형 회귀와 달리 s 자 함수를 사용하여 참(1) 거짓(0)을 분류한다.

시그모이드 함수 : 로지스틱 회귀에서 사용하는 s자 함수. 
시드모이드 함수는 x의 값이 커지면 y의 값은 1에 근사하게 되고 x의 값이 작아지면 y의 값은 0에 근사하게 되어 s자 형태의 그래프가 된다. 
이진 분류에 많이 사용된다.

로지스틱 회귀 모델의 성능 평가 지표는 이진 분류 결과를 평가하기 위해 오차 행렬에 기반한 성능 지표인 정밀도(예측이 Positive중 참인 비율), 재현율(실제가 positive중 참), f1 스코어(정밀도와 재현율 결합), ROC_AUC(오차행렬의 FP Rate가 변할 때 TP Rate가 어떻게 변하는지 나타내는 곡선) 1에 가까울수록 좋다)를 사용한다. 



In [1]:
# 유방암 진단 데이터셋

from sklearn.datasets import load_breast_cancer
b_cancer  = load_breast_cancer()

In [53]:
b_cancer.DESCR # 데이터 셋에 대한 설명 확인 
b_cancer_df = pd.DataFrame(b_cancer.data, columns = b_cancer.feature_names)
# 데이터셋 객체의 datㅁ 배열 b_cancer.data 즉 독립변수 x가 되는 피처를 dataFrame 자료형으로 변환하여 b_cancer_df 생성
b_cancer_df['diagnosis'] = b_cancer.target
# 유방암 유무 class로 사용할 diagnosis 컬럼을 b_cancer_df에 추가하고 데이터셋 객체의 ㅅarget 컬럼 b_cancer.target 저장한다
b_cancer_df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,diagnosis
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [54]:
# 데이터셋 크기 (데이터 샘플, 열 갯수)
b_cancer_df.shape 
b_cancer_df.info # 정보 확인 악성이면 1 양성이면 0 

<bound method DataFrame.info of      mean radius  mean texture  mean perimeter  mean area  mean smoothness  \
0          17.99         10.38          122.80     1001.0          0.11840   
1          20.57         17.77          132.90     1326.0          0.08474   
2          19.69         21.25          130.00     1203.0          0.10960   
3          11.42         20.38           77.58      386.1          0.14250   
4          20.29         14.34          135.10     1297.0          0.10030   
..           ...           ...             ...        ...              ...   
564        21.56         22.39          142.00     1479.0          0.11100   
565        20.13         28.25          131.20     1261.0          0.09780   
566        16.60         28.08          108.30      858.1          0.08455   
567        20.60         29.33          140.10     1265.0          0.11780   
568         7.76         24.54           47.92      181.0          0.05263   

     mean compactness  mean con

In [3]:
# 로지스틱 회귀 분석의 피처로 사용할 데이터를 평균이 0 분산이 1이 되는 정규 분포 형태로 맞춘다. 

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler

b_cancer_scaled = scaler.fit_transform(b_cancer.data)# 피처로 사용할 데이터에 대해 정규 분포 스케일링을 수행하여 저장한다. 
print(b_cancer.data[0]) 
print(b_cancer_scaled[0]) # 값이 조정된거 확인 


# 로지스틱 회귀를 이용하여 분석 모델 구축하기 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# x,y 설정하기 
y = b_cancer_df['diagnosis']
x = b_cancer_scaled # 정규 분포로 스케일링 

# 훈련용 데이터와 평가용 데이터 분할하기
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3,random_state = 0)

# 로지스틱 회귀 분석 : 모델 객체 생성
lr_b_cancer = LogisticRegression() 

# 로지스틱 회귀 분석 : 모델 훈련 학습 데이터로 모델 학습을 수행(fit)한다.
lr_b_cancer.fit(x_train, y_train)

# 로지스틱 회귀 분석 평가 데이터에 대한 예측 수행. x_test를 가지고 예측을 수행predict 수행하여 예측값 y_predict를 구한다)
y_predict = lr_b_cancer.predict(x_test)

# 생성한 모델의 성능 확인하기

from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score 
# 평가 지수 계산을 제공하는 모듈 임포트

# 오차 행렬
# 평가를 위해 7:3으로 분할한 test데이터에 대해 이진 분류의 성능 평가 기본이 되는 오차 행렬을 구한다. 
confusion_matrix(y_test,y_predict)

# 성능 평가 지표인 정확도, 정밀도, 재현율, F1스코어, ROC-AUC 스코어를 구한다.
accuracy = accuracy_score(y_test,y_predict)
precision = precision_score(y_test,y_predict)
recall = recall_score(y_test,y_predict)
f1 = f1_score(y_test,y_predict)
roc_auc = roc_auc_score(y_test,y_predict)
print('정확도:{},정밀도:{},재현율:{},F1:{}'.format(accuracy,precision,recall,f1))

print('ROC_AUC:',roc_auc)
